In [1]:
"""
🚦 按车流比例进行交通信号灯通行时间分配优化（含详细注释）

目标：在规定的绿灯总时长下，按照各类车辆的实际车流比例分配通行时间，
并通过线性规划尽可能提高总通行车辆数量，同时满足比例误差约束。
"""

import numpy as np
import pandas as pd
from scipy.optimize import linprog

# ------------------------------
# 🚗 定义车辆类型与通行效率（单位：辆/秒）
vehicle_types = ["小汽车", "卡车", "公交车", "自行车"]
efficiency = np.array([1.5, 1.0, 0.8, 2.0])  # 每种车每秒可以通过的辆数
total_time = 60  # 总信号灯通行时间，单位：秒

# ------------------------------
# 🎯 构建目标函数（最大化通行车辆总数）
# linprog 默认是最小化，所以我们取负数来转化为“最大化”
c = -efficiency

# ------------------------------
# 📊 设定各类车辆的期望通行时间比例（来源于交通调查）
# 如小汽车占 40%，卡车20%，公交车20%，自行车20%
ratio = np.array([0.4, 0.2, 0.2, 0.2])

# ------------------------------
# ⚖️ 构建比例容差约束（±5秒）
# 构造一系列不等式，限制每类车的通行时间不要偏离目标值超过 ±ε 秒
epsilon = 5  # 允许的最大偏差（秒）
A_ub = []  # 不等式左侧系数矩阵
b_ub = []  # 不等式右侧值

for i in range(len(vehicle_types)):
    # 上界：x_i <= ratio_i * total_time + epsilon
    row_upper = [0] * len(vehicle_types)
    row_upper[i] = 1
    A_ub.append(row_upper)
    b_ub.append(ratio[i] * total_time + epsilon)

    # 下界：-x_i <= -ratio_i * total_time + epsilon  ==> x_i >= ratio_i * total_time - epsilon
    row_lower = [0] * len(vehicle_types)
    row_lower[i] = -1
    A_ub.append(row_lower)
    b_ub.append(-ratio[i] * total_time + epsilon)

# ------------------------------
# ⏱ 添加总通行时间等式约束：所有车辆通行时间加起来必须等于 total_time
A_eq = [np.ones_like(efficiency)]
b_eq = [total_time]

# ------------------------------
# 🚫 设置每种车辆通行时间下限（不少于 5 秒）
bounds = [(5, None) for _ in vehicle_types]

# ------------------------------
# ✅ 求解线性规划模型
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

# ------------------------------
# 📋 输出结果
result_df = pd.DataFrame({
    "车辆类型": vehicle_types,
    "推荐通行时间（秒）": res.x,
    "单位效率（辆/秒）": efficiency,
    "预计通行辆数": res.x * efficiency,
    "比例目标（秒）": ratio * total_time,
    "偏差（实际 - 目标）": res.x - (ratio * total_time)
})
result_df.loc["总计"] = ["", res.x.sum(), "", (res.x * efficiency).sum(), "", ""]

# 显示表格
print("\n🚥 按比例约束下的信号灯通行优化结果：\n")
print(result_df)



🚥 按比例约束下的信号灯通行优化结果：

   车辆类型  推荐通行时间（秒） 单位效率（辆/秒）  预计通行辆数 比例目标（秒） 偏差（实际 - 目标）
0   小汽车       29.0       1.5    43.5    24.0         5.0
1    卡车        7.0       1.0     7.0    12.0        -5.0
2   公交车        7.0       0.8     5.6    12.0        -5.0
3   自行车       17.0       2.0    34.0    12.0         5.0
总计            60.0              90.1                    
